<a href="https://colab.research.google.com/github/voshna123/tensorflow_practice/blob/main/practice5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-01-08 03:26:31--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  40.4MB/s    in 4.6s    

2025-01-08 03:26:36 (35.0 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
import zipfile

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [16]:
base = tf.keras.applications.EfficientNetB0()
base.trainable = False

In [20]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.Resizing(224,224),
    tf.keras.layers.RandomFlip("Horizontal"),
])

In [22]:
inputs = tf.keras.layers.Input(shape = (224,224,3))

x = data_aug(inputs)

x = base(x, training = False)

x= tf.keras.layers.GlobalAveragePooling2D()(x)

outputs = tf.keras.layers.Dense(10, activation = 'softmax')(x)

model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [23]:
train_dir = "/content/10_food_classes_10_percent/train"
test_dir ="/content/10_food_classes_10_percent/test"

In [24]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle = True,
    label_mode = "categorical",
    image_size = (224,224),
    seed= 42
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    shuffle = True,
    label_mode = "categorical",
    image_size = (224,224)
)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [25]:
path = "model_weight.weights.h5"

callback = tf.keras.callbacks.ModelCheckpoint(path,
                                              save_best_only = True,
                                              save_weights_only=True)

In [26]:
hist_1 = model_1.fit(train_data,
                     epochs = 5,
                     validation_data = test_data,
                     callbacks =[callback])

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.2392 - loss: 2.1472 - val_accuracy: 0.6720 - val_loss: 1.4092
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 42s 338ms/step - accuracy: 0.7053 - loss: 1.2689 - val_accuracy: 0.7712 - val_loss: 0.9821
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 371ms/step - accuracy: 0.7849 - loss: 0.9182 - val_accuracy: 0.8000 - val_loss: 0.8068
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 301ms/step - accuracy: 0.8353 - loss: 0.7405 - val_accuracy: 0.8160 - val_loss: 0.7139
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 525ms/step - accuracy: 0.8464 - loss: 0.6264 - val_accuracy: 0.8212 - val_loss: 0.6578


In [39]:
for layer in base.layers[:-20]:
  layer.trainable = False

for layer in base.layers[-20:]:
  layer.trainable = True

In [40]:
model_1.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_1.load_weights("/content/model_weight.weights.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
for i,layer in enumerate(model_1.layers[2].layers):
  print(i,layer, layer.trainable)

0 <InputLayer name=input_layer_16, built=True> False
1 <Rescaling name=rescaling_10, built=True> False
2 <Normalization name=normalization_5, built=True> False
3 <Rescaling name=rescaling_11, built=True> False
4 <ZeroPadding2D name=stem_conv_pad, built=True> False
5 <Conv2D name=stem_conv, built=True> False
6 <BatchNormalization name=stem_bn, built=True> False
7 <Activation name=stem_activation, built=True> False
8 <DepthwiseConv2D name=block1a_dwconv, built=True> False
9 <BatchNormalization name=block1a_bn, built=True> False
10 <Activation name=block1a_activation, built=True> False
11 <GlobalAveragePooling2D name=block1a_se_squeeze, built=True> False
12 <Reshape name=block1a_se_reshape, built=True> False
13 <Conv2D name=block1a_se_reduce, built=True> False
14 <Conv2D name=block1a_se_expand, built=True> False
15 <Multiply name=block1a_se_excite, built=True> False
16 <Conv2D name=block1a_project_conv, built=True> False
17 <BatchNormalization name=block1a_project_bn, built=True> False
18

In [43]:
hist_2 = model_1.fit(train_data,
                     epochs = 15,
                     validation_data = test_data,
                     initial_epoch = 5)

Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.8251 - loss: 0.7537 - val_accuracy: 0.8492 - val_loss: 0.4936
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 36s 284ms/step - accuracy: 0.9758 - loss: 0.1324 - val_accuracy: 0.8500 - val_loss: 0.4904
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 330ms/step - accuracy: 0.9897 - loss: 0.0559 - val_accuracy: 0.8528 - val_loss: 0.5032
Epoch 9/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 289ms/step - accuracy: 0.9936 - loss: 0.0388 - val_accuracy: 0.8812 - val_loss: 0.3803
Epoch 10/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 290ms/step - accuracy: 0.9937 - loss: 0.0248 - val_accuracy: 0.8828 - val_loss: 0.3789
Epoch 11/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 319ms/step - accuracy: 0.9976 - loss: 0.0205 - val_accuracy: 0.8800 - val_loss: 0.3852
Epoch 12/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 285ms/step - accuracy: 0.9997 - loss: 0.0123 - val_accuracy: 0.8792 - val_loss: 0.3940
Epoch 13/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 253ms/step - accuracy: 0.9931 - loss: 0.0223 - val_accura

In [44]:
for layer in base.layers[-30:]:
  layer.trainable = True

In [45]:
model_1.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_1.load_weights("/content/model_weight.weights.h5")

In [46]:
for i,layer in enumerate(model_1.layers[2].layers):
  print(i,layer, layer.trainable)

0 <InputLayer name=input_layer_16, built=True> False
1 <Rescaling name=rescaling_10, built=True> False
2 <Normalization name=normalization_5, built=True> False
3 <Rescaling name=rescaling_11, built=True> False
4 <ZeroPadding2D name=stem_conv_pad, built=True> False
5 <Conv2D name=stem_conv, built=True> False
6 <BatchNormalization name=stem_bn, built=True> False
7 <Activation name=stem_activation, built=True> False
8 <DepthwiseConv2D name=block1a_dwconv, built=True> False
9 <BatchNormalization name=block1a_bn, built=True> False
10 <Activation name=block1a_activation, built=True> False
11 <GlobalAveragePooling2D name=block1a_se_squeeze, built=True> False
12 <Reshape name=block1a_se_reshape, built=True> False
13 <Conv2D name=block1a_se_reduce, built=True> False
14 <Conv2D name=block1a_se_expand, built=True> False
15 <Multiply name=block1a_se_excite, built=True> False
16 <Conv2D name=block1a_project_conv, built=True> False
17 <BatchNormalization name=block1a_project_bn, built=True> False
18

In [47]:
hist_2 = model_1.fit(train_data,
                     epochs = 15,
                     validation_data = test_data,
                     initial_epoch = 5)

Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.8000 - loss: 0.8018 - val_accuracy: 0.8224 - val_loss: 0.5577
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 35s 327ms/step - accuracy: 0.9608 - loss: 0.1741 - val_accuracy: 0.8384 - val_loss: 0.5841
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 534ms/step - accuracy: 0.9883 - loss: 0.0526 - val_accuracy: 0.8716 - val_loss: 0.4351
Epoch 9/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 15s 276ms/step - accuracy: 0.9930 - loss: 0.0300 - val_accuracy: 0.8772 - val_loss: 0.4017
Epoch 10/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 323ms/step - accuracy: 0.9992 - loss: 0.0225 - val_accuracy: 0.8832 - val_loss: 0.3764
Epoch 11/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 326ms/step - accuracy: 0.9983 - loss: 0.0191 - val_accuracy: 0.8844 - val_loss: 0.3901
Epoch 12/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 284ms/step - accuracy: 0.9992 - loss: 0.0119 - val_accuracy: 0.8892 - val_loss: 0.3686
Epoch 13/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 304ms/step - accuracy: 0.9968 - loss: 0.0220 - val_accura